In [1]:
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import RandomizedSearchCV

In [2]:
file=r'/Users/lalitsachan/Dropbox/0.0 Data/census_income.csv'

cd= pd.read_csv(file)

In [3]:
cd['Y'].unique()

array([' <=50K', ' >50K'], dtype=object)

In [4]:
cd['Y']=(cd['Y']==' >50K').astype(int)

In [5]:
pd.crosstab(cd['education'],cd['education.num'])

education.num,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
education,,,,,,,,,,,,,,,,
10th,0,0,0,0,0,933,0,0,0,0,0,0,0,0,0,0
11th,0,0,0,0,0,0,1175,0,0,0,0,0,0,0,0,0
12th,0,0,0,0,0,0,0,433,0,0,0,0,0,0,0,0
1st-4th,0,168,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5th-6th,0,0,333,0,0,0,0,0,0,0,0,0,0,0,0,0
7th-8th,0,0,0,646,0,0,0,0,0,0,0,0,0,0,0,0
9th,0,0,0,0,514,0,0,0,0,0,0,0,0,0,0,0
Assoc-acdm,0,0,0,0,0,0,0,0,0,0,0,1067,0,0,0,0
Assoc-voc,0,0,0,0,0,0,0,0,0,0,1382,0,0,0,0,0


In [6]:
del cd['education']

In [7]:
cd.head()

,age,workclass,fnlwgt,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,Y
0,39,State-gov,77516,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,215646,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,234721,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,338409,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0


In [8]:
cd.select_dtypes(['object']).nunique()

workclass          9
marital.status     7
occupation        15
relationship       6
race               5
sex                2
native.country    42
dtype: int64

In [9]:
cd['occupation'].value_counts()

 Prof-specialty       4140
 Craft-repair         4099
 Exec-managerial      4066
 Adm-clerical         3770
 Sales                3650
 Other-service        3295
 Machine-op-inspct    2002
 ?                    1843
 Transport-moving     1597
 Handlers-cleaners    1370
 Farming-fishing       994
 Tech-support          928
 Protective-serv       649
 Priv-house-serv       149
 Armed-Forces            9
Name: occupation, dtype: int64

In [10]:
cd['native.country'].value_counts()

 United-States                 29170
 Mexico                          643
 ?                               583
 Philippines                     198
 Germany                         137
 Canada                          121
 Puerto-Rico                     114
 El-Salvador                     106
 India                           100
 Cuba                             95
 England                          90
 Jamaica                          81
 South                            80
 China                            75
 Italy                            73
 Dominican-Republic               70
 Vietnam                          67
 Guatemala                        64
 Japan                            62
 Poland                           60
 Columbia                         59
 Taiwan                           51
 Haiti                            44
 Iran                             43
 Portugal                         37
 Nicaragua                        34
 Peru                             31
 

In [11]:
cat_cols=cd.select_dtypes(['object']).columns

In [12]:
cat_cols

Index(['workclass', 'marital.status', 'occupation', 'relationship', 'race',
       'sex', 'native.country'],
      dtype='object')

In [13]:
for col in cat_cols:
    freqs=cd[col].value_counts()
    k=freqs.index[freqs>99][:-1]
    for cat in k:
        name=col+'_'+cat
        cd[name]=(cd[col]==cat).astype(int)
    del cd[col]
    print(col)

workclass
marital.status
occupation
relationship
race
sex
native.country


In [14]:
cd.shape

(32561, 49)

In [15]:
x_train=cd.drop(['Y'],axis=1)
y_train=cd['Y']

In [16]:
help(MLPClassifier)

Help on class MLPClassifier in module sklearn.neural_network.multilayer_perceptron:

class MLPClassifier(BaseMultilayerPerceptron, sklearn.base.ClassifierMixin)
 |  Multi-layer Perceptron classifier.
 |  
 |  This model optimizes the log-loss function using LBFGS or stochastic
 |  gradient descent.
 |  
 |  .. versionadded:: 0.18
 |  
 |  Parameters
 |  ----------
 |  hidden_layer_sizes : tuple, length = n_layers - 2, default (100,)
 |      The ith element represents the number of neurons in the ith
 |      hidden layer.
 |  
 |  activation : {'identity', 'logistic', 'tanh', 'relu'}, default 'relu'
 |      Activation function for the hidden layer.
 |  
 |      - 'identity', no-op activation, useful to implement linear bottleneck,
 |        returns f(x) = x
 |  
 |      - 'logistic', the logistic sigmoid function,
 |        returns f(x) = 1 / (1 + exp(-x)).
 |  
 |      - 'tanh', the hyperbolic tan function,
 |        returns f(x) = tanh(x).
 |  
 |      - 'relu', the rectified linear u

In [17]:
parameters={
'learning_rate': ["constant", "invscaling", "adaptive"],
'hidden_layer_sizes': [(5,10,5),(20,10),(10,20)],
'alpha': [0.3,.1,.01],
'activation': ["logistic", "relu", "tanh"]
}

In [18]:
clf=MLPClassifier()

In [19]:
random_search=RandomizedSearchCV(clf,n_iter=5,cv=10,param_distributions=parameters,
                                 scoring='roc_auc',random_state=2,n_jobs=-1,verbose=20)

In [20]:
random_search.fit(x_train,y_train)

Fitting 10 folds for each of 5 candidates, totalling 50 fits
[CV] learning_rate=invscaling, hidden_layer_sizes=(10, 20), alpha=0.01, activation=logistic 
[CV] learning_rate=invscaling, hidden_layer_sizes=(10, 20), alpha=0.01, activation=logistic 
[CV] learning_rate=invscaling, hidden_layer_sizes=(10, 20), alpha=0.01, activation=logistic 
[CV] learning_rate=invscaling, hidden_layer_sizes=(10, 20), alpha=0.01, activation=logistic 
[CV]  learning_rate=invscaling, hidden_layer_sizes=(10, 20), alpha=0.01, activation=logistic, score=0.5017093356419929, total=   2.1s
[CV] learning_rate=invscaling, hidden_layer_sizes=(10, 20), alpha=0.01, activation=logistic 
[CV]  learning_rate=invscaling, hidden_layer_sizes=(10, 20), alpha=0.01, activation=logistic, score=0.5006377551020408, total=   2.0s
[CV] learning_rate=invscaling, hidden_layer_sizes=(10, 20), alpha=0.01, activation=logistic 


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    2.6s


[CV]  learning_rate=invscaling, hidden_layer_sizes=(10, 20), alpha=0.01, activation=logistic, score=0.5001406053926425, total=   3.0s
[CV] learning_rate=invscaling, hidden_layer_sizes=(10, 20), alpha=0.01, activation=logistic 


[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    3.5s


[CV]  learning_rate=invscaling, hidden_layer_sizes=(10, 20), alpha=0.01, activation=logistic, score=0.5, total=   3.2s
[CV] learning_rate=invscaling, hidden_layer_sizes=(10, 20), alpha=0.01, activation=logistic 


...

[CV]  learning_rate=adaptive, hidden_layer_sizes=(5, 10, 5), alpha=0.01, activation=tanh, score=0.5, total=   2.5s
[CV] learning_rate=adaptive, hidden_layer_sizes=(5, 10, 5), alpha=0.01, activation=tanh 
[CV]  learning_rate=adaptive, hidden_layer_sizes=(5, 10, 5), alpha=0.01, activation=tanh, score=0.5006377551020408, total=   1.8s
[CV] learning_rate=adaptive, hidden_layer_sizes=(5, 10, 5), alpha=0.01, activation=tanh 
[CV]  learning_rate=adaptive, hidden_layer_sizes=(5, 10, 5), alpha=0.01, activation=tanh, score=0.5108199074532725, total=   2.7s
[CV] learning_rate=adaptive, hidden_layer_sizes=(5, 10, 5), alpha=0.01, activation=tanh 


[Parallel(n_jobs=-1)]: Done  46 out of  50 | elapsed:   35.4s remaining:    3.1s


[CV]  learning_rate=adaptive, hidden_layer_sizes=(5, 10, 5), alpha=0.01, activation=tanh, score=0.5308862319199524, total=   2.2s
[CV]  learning_rate=adaptive, hidden_layer_sizes=(5, 10, 5), alpha=0.01, activation=tanh, score=0.5019132653061225, total=   2.0s
[CV]  learning_rate=adaptive, hidden_layer_sizes=(5, 10, 5), alpha=0.01, activation=tanh, score=0.5694428104979856, total=   2.9s
[CV]  learning_rate=adaptive, hidden_layer_sizes=(5, 10, 5), alpha=0.01, activation=tanh, score=0.4987456451027013, total=   1.3s


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   36.8s finished


RandomizedSearchCV(cv=10, error_score='raise',
          estimator=MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False),
          fit_params=None, iid=True, n_iter=5, n_jobs=-1,
          param_distributions={'learning_rate': ['constant', 'invscaling', 'adaptive'], 'hidden_layer_sizes': [(5, 10, 5), (20, 10), (10, 20)], 'alpha': [0.3, 0.1, 0.01], 'activation': ['logistic', 'relu', 'tanh']},
          pre_dispatch='2*n_jobs', random_state=2, refit=True,
          return_train_score='warn', scoring='roc_auc', verbose=20)

In [21]:
random_search.best_estimator_

MLPClassifier(activation='relu', alpha=0.3, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(20, 10), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [22]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.5f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [23]:
report(random_search.cv_results_,5)

Model with rank: 1
Mean validation score: 0.618 (std: 0.04654)
Parameters: {'learning_rate': 'constant', 'hidden_layer_sizes': (20, 10), 'alpha': 0.3, 'activation': 'relu'}

Model with rank: 2
Mean validation score: 0.592 (std: 0.05016)
Parameters: {'learning_rate': 'adaptive', 'hidden_layer_sizes': (5, 10, 5), 'alpha': 0.3, 'activation': 'relu'}

Model with rank: 3
Mean validation score: 0.515 (std: 0.01623)
Parameters: {'learning_rate': 'constant', 'hidden_layer_sizes': (10, 20), 'alpha': 0.01, 'activation': 'tanh'}

Model with rank: 4
Mean validation score: 0.514 (std: 0.02177)
Parameters: {'learning_rate': 'adaptive', 'hidden_layer_sizes': (5, 10, 5), 'alpha': 0.01, 'activation': 'tanh'}

Model with rank: 5
Mean validation score: 0.503 (std: 0.00552)
Parameters: {'learning_rate': 'invscaling', 'hidden_layer_sizes': (10, 20), 'alpha': 0.01, 'activation': 'logistic'}



In [24]:
mlp=random_search.best_estimator_

In [25]:
mlp.fit(x_train,y_train)

MLPClassifier(activation='relu', alpha=0.3, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(20, 10), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)